In [1]:
# Importing required libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv(r"""https://raw.githubusercontent.com/shrikant-temburwar/Wine-Quality-Dataset/master/winequality-red.csv""", sep=';')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [5]:
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [7]:
df.duplicated().sum()

240

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.duplicated().sum()

0

In [10]:
X = df.drop(['quality'], axis=1)
y = df['quality']

In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=10)

In [12]:
!nvidia-smi

Thu Nov 24 15:32:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Gradient Boosting Classifier**

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
model_dbm = GradientBoostingClassifier()

In [16]:
model_dbm.fit(X_train, y_train)

GradientBoostingClassifier()

In [17]:
model_dbm.score(X_train, y_train)

0.9153846153846154

In [18]:
y_pred_dbm = model_dbm.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score

In [23]:
accuracy_score(y_test, y_pred_dbm)

0.5746102449888641

# **XGBoost Algo**

In [26]:
from xgboost import XGBClassifier

In [27]:
model_xgb = XGBClassifier()

In [28]:
model_xgb.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob')

In [29]:
model_xgb.score(X_train, y_train)

0.8131868131868132

In [30]:
y_pred_xgb = model_xgb.predict(X_test)

In [31]:
accuracy_score(y_test, y_pred_xgb)

0.5590200445434298

# **HyperParameter Tuning with Gradient Boosting Classifier**

In [33]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [43]:
grid_param = {
    'learning_rate': [0.001,0.0001],
    'n_estimators': [90,115],
    'criterion': ['friedman_mse', 'mse'],
    'max_depth': [8,13,17,18,20],
    'min_samples_leaf': range(4,6,1),
    'min_samples_split': range(2,10,2),
    'max_features': ['auto', 'log2']
}

In [44]:
tuned_model_gbm = HalvingGridSearchCV(estimator=model_dbm, param_grid=grid_param, cv=3 , verbose=1, n_jobs=-1)

In [45]:
tuned_model_gbm.fit(X_train, y_train)

n_iterations: 3
n_required_iterations: 6
n_possible_iterations: 3
min_resources_: 36
max_resources_: 910
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 640
n_resources: 36
Fitting 3 folds for each of 640 candidates, totalling 1920 fits
----------
iter: 1
n_candidates: 214
n_resources: 108
Fitting 3 folds for each of 214 candidates, totalling 642 fits
----------
iter: 2
n_candidates: 72
n_resources: 324
Fitting 3 folds for each of 72 candidates, totalling 216 fits


HalvingGridSearchCV(cv=3, estimator=GradientBoostingClassifier(), n_jobs=-1,
                    param_grid={'criterion': ['friedman_mse', 'mse'],
                                'learning_rate': [0.001, 0.0001],
                                'max_depth': [8, 13, 17, 18, 20],
                                'max_features': ['auto', 'log2'],
                                'min_samples_leaf': range(4, 6),
                                'min_samples_split': range(2, 10, 2),
                                'n_estimators': [90, 115]},
                    verbose=1)

In [47]:
tuned_model_gbm.best_params_

{'criterion': 'friedman_mse',
 'learning_rate': 0.001,
 'max_depth': 20,
 'max_features': 'log2',
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 115}

In [48]:
tuned_y_pred = tuned_model_gbm.predict(X_test)

In [49]:
accuracy_score(y_test, tuned_y_pred)

0.5545657015590201

In [50]:
# (criterion= 'friedman_mse',
#  learning_rate= 0.001,
#  max_depth= 20,
#  max_features= 'log2',
#  min_samples_leaf= 4,
#  min_samples_split= 8,
#  n_estimators= 115)

In [52]:
model_dbm_tuned = GradientBoostingClassifier(criterion= 'friedman_mse',
 learning_rate= 0.001,
 max_depth= 20,
 max_features= 'log2',
 min_samples_leaf= 4,
 min_samples_split= 8,
 n_estimators= 115)

In [53]:
model_dbm_tuned.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.001, max_depth=20,
                           max_features='log2', min_samples_leaf=4,
                           min_samples_split=8, n_estimators=115)

In [54]:
y_pred_tuned_gbc = model_dbm_tuned.predict(X_test)

In [56]:
accuracy_score(y_test, y_pred_tuned_gbc)

0.5478841870824054

In [58]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 37.5 MB/s 
     |████████████████████████████████| 81 kB 11.2 MB/s 
     |████████████████████████████████| 209 kB 73.4 MB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 147 kB 73.9 MB/s 
     |████████████████████████████████| 112 kB 70.4 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=c346bcb13291145eb14e0fd5762b60c1c78f909e3f87e7fadf45131c2d5d891b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [60]:
def objective_classification(trial,data=X,target=y) :
    train_x , test_x , train_y , test_y= train_test_split(data , target , test_size = .20 ,random_state=10)
    
    param  = {
        'tree_method' :'gpu_hist',
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy
        

In [62]:
import optuna

xgb_classification_oputuna = optuna.create_study()

[I 2022-11-24 17:43:48,214] A new study created in memory with name: no-name-b4f4ba2f-0613-4de6-94f0-d9434a5f86be


In [ ]:
xgb_classification_oputuna.optimize(objective_classification , n_trials = 100 )

[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[0]	validation_0-merror:0.522059
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[1]	validation_0-merror:0.455882
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[2]	validation_0-merror:0.466912
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[3]	validation_0-merror:0.459559
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[4]	validation_0-merror:0.444853
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[5]	validation_0-merror:0.448529
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[6]	validation_0-merror:0.430147
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[7]	validation_0-merror:0.441176
[17:43:56] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[8]	validation_0-merror:0.444853
[17:43:56] INFO: /w

[I 2022-11-24 17:44:00,377] Trial 0 finished with value: 0.5551470588235294 and parameters: {'booster': 'dart', 'lambda': 0.10709334207294503, 'alpha': 0.5553369153071606, 'subsample': 0.47177041682568144, 'colsample_bytree': 0.29552944312760276}. Best is trial 0 with value: 0.5551470588235294.


[17:44:00] ======== Monitor: Learner ========
[17:44:00] EvalOneIter: 0.31943s, 100 calls @ 3194us
[17:44:00] GetGradient: 0.00727896s, 100 calls @ 72us
[17:44:00] PredictRaw: 1.19238s, 100 calls @ 11923us
[17:44:00] UpdateOneIter: 3.23956s, 100 calls @ 32395us
[17:44:00] ======== Monitor: GBTree ========
[17:44:00] BoostNewTrees: 2.01549s, 100 calls @ 20154us
[17:44:00] CommitModel: 0.0239619s, 100 calls @ 239us
[17:44:00] ======== Monitor:  ========
[17:44:00] ======== Monitor: updater_gpu_hist ========
[17:44:00] BinningCompression: 0.000398735s, 1 calls @ 398us
[17:44:00] InitData: 0.0334258s, 600 calls @ 55us
[17:44:00] InitDataOnce: 0.0333104s, 1 calls @ 33310us
[17:44:00] Quantiles: 0.00217888s, 1 calls @ 2178us
[17:44:00] Update: 1.99144s, 600 calls @ 3319us
[17:44:00] ======== NCCL Statistics========
[17:44:00] AllReduce calls: 2857
[17:44:00] AllReduce total MB communicated: 42
[17:44:00] ======== Monitor: HistCutMatrix ========
[17:44:00] ======== Monitor: DeviceShard0 =====

[I 2022-11-24 17:44:00,790] Trial 1 finished with value: 0.4963235294117647 and parameters: {'booster': 'gblinear', 'lambda': 0.8215626168439611, 'alpha': 0.8483608941748045, 'subsample': 0.44234111517489316, 'colsample_bytree': 0.3308836409973136}. Best is trial 1 with value: 0.4963235294117647.


[17:44:00] EvalOneIter: 0.0121946s, 100 calls @ 121us
[17:44:00] GetGradient: 0.00871802s, 100 calls @ 87us
[17:44:00] PredictRaw: 0.00153146s, 100 calls @ 15us
[17:44:00] UpdateOneIter: 0.215132s, 100 calls @ 2151us
[17:44:00] ======== Monitor: GBLinear ========
[17:44:00] DoBoost: 0.204551s, 100 calls @ 2045us
[17:44:00] PredictBatch: 0.00475812s, 202 calls @ 23us
[17:44:00] PredictBatchInternal: 0.136893s, 203 calls @ 674us
[0]	validation_0-merror:0.5625
[1]	validation_0-merror:0.5625
[2]	validation_0-merror:0.555147
[3]	validation_0-merror:0.569853
[4]	validation_0-merror:0.566176
[5]	validation_0-merror:0.577206
[6]	validation_0-merror:0.573529
[7]	validation_0-merror:0.558824
[8]	validation_0-merror:0.566176
[9]	validation_0-merror:0.547794
[10]	validation_0-merror:0.544118
[11]	validation_0-merror:0.540441
[12]	validation_0-merror:0.533088
[13]	validation_0-merror:0.533088
[14]	validation_0-merror:0.536765
[15]	validation_0-merror:0.533088
[16]	validation_0-merror:0.533088
[17]	

...

[17:45:14] ======== Monitor: Learner ========
[17:45:14] EvalOneIter: 0.00755109s, 100 calls @ 75us
[17:45:14] GetGradient: 0.00480378s, 100 calls @ 48us
[17:45:14] PredictRaw: 0.00293488s, 100 calls @ 29us
[17:45:14] UpdateOneIter: 2.04804s, 100 calls @ 20480us
[17:45:14] ======== Monitor: GBTree ========
[17:45:14] BoostNewTrees: 2.0301s, 100 calls @ 20300us
[17:45:14] CommitModel: 0.00974867s, 100 calls @ 97us
[17:45:14] ======== Monitor:  ========
[17:45:14] DevicePredictInternal: 0.0100011s, 202 calls @ 49us
[17:45:14] PredictFromCache: 0.003817s, 199 calls @ 19us
[17:45:14] ======== Monitor: updater_gpu_hist ========
[17:45:14] BinningCompression: 0.000422893s, 1 calls @ 422us
[17:45:14] InitData: 0.0159305s, 600 calls @ 26us
[17:45:14] InitDataOnce: 0.0157806s, 1 calls @ 15780us
[17:45:14] Quantiles: 0.00216505s, 1 calls @ 2165us
[17:45:14] Update: 2.00268s, 600 calls @ 3337us
[17:45:14] ======== NCCL Statistics========
[17:45:14] AllReduce calls: 2818
[17:45:14] AllReduce total

[I 2022-11-24 17:45:15,411] Trial 71 finished with value: 0.4963235294117647 and parameters: {'booster': 'gblinear', 'lambda': 0.7725894868974071, 'alpha': 0.4548204570643642, 'subsample': 0.4838933519025652, 'colsample_bytree': 0.21137976047938545}. Best is trial 1 with value: 0.4963235294117647.


[0]	validation_0-merror:0.5625
[1]	validation_0-merror:0.5625
[2]	validation_0-merror:0.555147
[3]	validation_0-merror:0.569853
[4]	validation_0-merror:0.566176
[5]	validation_0-merror:0.577206
[6]	validation_0-merror:0.573529
[7]	validation_0-merror:0.558824
[8]	validation_0-merror:0.566176
[9]	validation_0-merror:0.547794
[10]	validation_0-merror:0.544118
[11]	validation_0-merror:0.540441
[12]	validation_0-merror:0.533088
[13]	validation_0-merror:0.533088
[14]	validation_0-merror:0.536765
[15]	validation_0-merror:0.533088
[16]	validation_0-merror:0.533088
[17]	validation_0-merror:0.533088
[18]	validation_0-merror:0.533088
[19]	validation_0-merror:0.536765
[20]	validation_0-merror:0.536765
[21]	validation_0-merror:0.536765
[22]	validation_0-merror:0.533088
[23]	validation_0-merror:0.533088
[24]	validation_0-merror:0.529412
[25]	validation_0-merror:0.529412
[26]	validation_0-merror:0.525735
[27]	validation_0-merror:0.525735
[28]	validation_0-merror:0.525735
[29]	validation_0-merror:0.5

[I 2022-11-24 17:45:15,870] Trial 72 finished with value: 0.4963235294117647 and parameters: {'booster': 'gblinear', 'lambda': 0.36176892292660545, 'alpha': 0.09799414527629656, 'subsample': 0.39126699360194744, 'colsample_bytree': 0.13262768987871748}. Best is trial 1 with value: 0.4963235294117647.



[17:45:15] PredictBatch: 0.00570716s, 202 calls @ 28us
[17:45:15] PredictBatchInternal: 0.118346s, 203 calls @ 582us
[0]	validation_0-merror:0.5625
[1]	validation_0-merror:0.5625
[2]	validation_0-merror:0.555147
[3]	validation_0-merror:0.569853
[4]	validation_0-merror:0.566176
[5]	validation_0-merror:0.577206
[6]	validation_0-merror:0.573529
[7]	validation_0-merror:0.558824
[8]	validation_0-merror:0.566176
[9]	validation_0-merror:0.547794
[10]	validation_0-merror:0.544118
[11]	validation_0-merror:0.540441
[12]	validation_0-merror:0.533088
[13]	validation_0-merror:0.533088
[14]	validation_0-merror:0.536765
[15]	validation_0-merror:0.533088
[16]	validation_0-merror:0.533088
[17]	validation_0-merror:0.533088
[18]	validation_0-merror:0.533088
[19]	validation_0-merror:0.536765
[20]	validation_0-merror:0.536765
[21]	validation_0-merror:0.536765
[22]	validation_0-merror:0.533088
[23]	validation_0-merror:0.533088
[24]	validation_0-merror:0.529412
[25]	validation_0-merror:0.529412
[26]	validat

In [64]:
xgb_classification_oputuna.best_params

{'booster': 'gblinear',
 'lambda': 0.8215626168439611,
 'alpha': 0.8483608941748045,
 'subsample': 0.44234111517489316,
 'colsample_bytree': 0.3308836409973136}

In [65]:
xgb_classification_oputuna.best_trial

FrozenTrial(number=1, values=[0.4963235294117647], datetime_start=datetime.datetime(2022, 11, 24, 17, 44, 0, 378711), datetime_complete=datetime.datetime(2022, 11, 24, 17, 44, 0, 789536), params={'booster': 'gblinear', 'lambda': 0.8215626168439611, 'alpha': 0.8483608941748045, 'subsample': 0.44234111517489316, 'colsample_bytree': 0.3308836409973136}, distributions={'booster': CategoricalDistribution(choices=('dart', 'gbtree', 'gblinear')), 'lambda': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'alpha': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'subsample': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'colsample_bytree': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)

In [66]:
param = xgb_classification_oputuna.best_trial.params

In [67]:
param

{'booster': 'gblinear',
 'lambda': 0.8215626168439611,
 'alpha': 0.8483608941748045,
 'subsample': 0.44234111517489316,
 'colsample_bytree': 0.3308836409973136}

In [69]:
xgb_final_class = XGBClassifier(**param)

In [72]:
xgb_final_class.fit(X_train , y_train)

XGBClassifier(alpha=0.8483608941748045, booster='gblinear',
              colsample_bytree=0.3308836409973136, lambda=0.8215626168439611,
              objective='multi:softprob', subsample=0.44234111517489316)

In [73]:
y_pred_best_params=xgb_final_class.predict(X_test)

In [74]:
accuracy_score(y_test,y_pred_best_params)

0.48106904231625836